In [1]:
import os
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import mutual_info_classif
from sklearn.ensemble import RandomForestClassifier

In [4]:
# Путь к папке с атаками
base_path = "../../distributions"

combined_path = "./combined_csvs/"
os.makedirs(combined_path, exist_ok=True)

distributions_folders = os.listdir(base_path)
distro_labels = { attack: idx + 1 for idx, attack in enumerate(distributions_folders) }
distro_labels

{'cauchy': 1, 'exponential': 2, 'normal': 3, 'gamma': 4}

In [5]:
# Найти все уникальные размеры выборок
packet_intervals = set()

for distro in distributions_folders:
    common_path = os.path.join(base_path, distro, "csvs")
    # print(common_path)
    # ../../distributions/cauchy/csvs
    # ../../distributions/exponential/csvs
    # ../../distributions/normal/csvs
    # ../../distributions/gamma/csvs
    for traffic_type in ["normal", "attack"]:
        traffic_path = os.path.join(common_path, traffic_type)
        for filename in os.listdir(traffic_path):
            # Извлечение размера выборки из имени файла с помощью регулярного выражения
            match = re.search(r"_(\d+)_\d+_", filename)
            if match:
                packet_intervals.add(int(match.group(1)))

# Сортируем размеры выборок
packet_intervals = sorted(packet_intervals)
print("Найденные размеры выборок пакетов:", packet_intervals)

../../distributions/cauchy/csvs
../../distributions/exponential/csvs
../../distributions/normal/csvs
../../distributions/gamma/csvs


In [9]:
# Объединение данных для каждого размера выборки
for packet_size in packet_intervals:
    combined_df = pd.DataFrame()

    for distro in distributions_folders:
        attack_path = os.path.join(base_path, distro, "csvs")

        # Определяем единственный файл нормального и атакующего трафика для текущего размера выборки
        normal_file = next(
                (
                        f
                        for f in os.listdir(os.path.join(attack_path, "normal"))
                        if f"_{packet_size}_0_" in f
                        ),
                None,
                )
        attack_file = next(
                (
                        f
                        for f in os.listdir(os.path.join(attack_path, "attack"))
                        if f"_{packet_size}_1_" in f
                        ),
                None,
                )

        if normal_file and attack_file:
            # Чтение файла с нормальным трафиком и добавление метки
            normal_df = pd.read_csv(os.path.join(attack_path, "normal", normal_file))
            normal_df["Label"] = 0  # Метка для нормального трафика
            combined_df = pd.concat([combined_df, normal_df], ignore_index=True)

            # Чтение файла с атакующим трафиком и добавление метки
            attack_df = pd.read_csv(os.path.join(attack_path, "attack", attack_file))
            attack_df["Label"] = distro_labels[distro]  # Уникальная метка для атаки
            combined_df = pd.concat([combined_df, attack_df], ignore_index=True)

    # Сохранение объединенного CSV для текущего размера выборки
    combined_file = os.path.join(combined_path, f"combined_{packet_size}_packets.csv")
    combined_df.to_csv(combined_file, index=False)